# LINE TO BUFFER

In [1]:
import geopandas as gpd
import rasterio
import numpy as np
from scipy.ndimage import map_coordinates
from osgeo import gdal


In [ ]:
# 读取河流线状数据
rivers = gpd.read_file('path_to_river_shapefile.shp')

# 读取DEM数据
with rasterio.open('path_to_dem.tif') as src:
    dem = src.read(1)  # 读取第一个波段
    affine = src.transform

In [ ]:
# 函数：从DEM获取高程
def sample_elevation(geom, dem, affine):
    coords = np.array([(x, y) for x, y in geom.coords])
    rc_coords = ~affine * (coords[:, 0], coords[:, 1])  # 转换坐标到像元空间
    rc_coords = np.round(rc_coords).astype(int)
    elevations = dem[rc_coords[1], rc_coords[0]]  # 采样高程
    return elevations

# 为每条河流采样高程
rivers['elevations'] = rivers.geometry.apply(lambda geom: sample_elevation(geom, dem, affine))
rivers['elevation_10m_above'] = rivers['elevations'].apply(lambda x: x + 10)

# 将加10米后的高程值转换为等值线需要的格式
with rasterio.open('dem_modified.tif', 'w', **src.meta) as dst:
    modified_dem = np.copy(dem)
    for idx, row in rivers.iterrows():
        for (x, y), val in zip(row.geometry.coords, row['elevation_10m_above']):
            rc = ~affine * (x, y)
            modified_dem[int(rc[1]), int(rc[0])] = val
    dst.write(modified_dem, 1)

# 使用GDAL生成等值线
gdal.DEMProcessing('contours.shp', 'dem_modified.tif', 'contour', contour_levels=[10])